# 시계열 딥러닝 모델을 사용한 충주댐 유입량 예측

## OBJECTIVE
- 대상지역 : 충주댐 상류
- 평가지표는 MAE, RMSE, R2 score를 사용
- 사용데이터는 직접 입력
- 최종결과물은 일단위 예측결과 csv 파일과 절차, 코드, 결과를 설명하는 ipynb 파일로 제출
## DATA
- 충주댐 상류의 유입량 데이터
  
- 충주댐 상류의 기상 데이터

1. 필요한 도구 호출, 도구 설정

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import warnings
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import r2_score

plt.rc('font', family='NanumBarunGothic')
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu") # cpu 사용시

OSError: [WinError 126] 지정된 모듈을 찾을 수 없습니다. Error loading "C:\Users\user\AppData\Roaming\Python\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

2. 데이터 불러오기

In [ ]:
df_asos=pd.read_parquet('https://github.com/Alsdnworks/lesson_practice/raw/main/ASOS_127.parquet') # 기상 데이터
rawdata = pd.read_parquet('https://github.com/Alsdnworks/lesson_practice/raw/main/ChungjuDam_info.parquet') # 유입량 데이터

3. 데이터 전처리

In [ ]:
target_column = 'Inflow' #유입량 데이터의 컬럼명

# 유입량 데이터 전처리
ym =pd.DataFrame()
ym['일시']=pd.date_range(start='1985-05-02', end='2024-01-01', freq='D')# 기간을 일별로 맞추기 위해 시작일부터 종료일까지의 날짜를 생성 (예: '2016-01-01' '2021-01-01')
rawdata['일시'] = pd.to_datetime(rawdata['일시'])
rawdata.columns = ['일시', target_column]
rawdata.fillna(method='bfill',inplace=True)

# ASOS 데이터 전처리
df_asos['일시'] = pd.to_datetime(df_asos['일시'])
#df_asos['DayRain_1']=df_asos['일강수량(mm)'].shift(1) # 1일 전 강수량 적용, 필요시 주석 해제
#df_asos['DayRain_2']=df_asos['일강수량(mm)'].shift(2) # 2일 전 강수량 적용, 필요시 주석 해제
df_asos.sort_values(by='일시',inplace=True)
df_asos.drop(['지점','지점명','기사'],axis=1,inplace=True)
for c in df_asos.columns:
    if '시각' in c:
        df_asos.drop(c,axis=1,inplace=True)
    if '강수' in c:
        df_asos.fillna(0,inplace=True)
df_asos.fillna(method='bfill',inplace=True)

# 유입량과 ASOS 데이터를 합침
dataset=pd.merge(df_asos,rawdata,on='일시',how='left')
dataset=pd.merge(ym,dataset,on='일시',how='left')
dataset.index = dataset['일시']
dataset.drop('일시',axis=1,inplace=True)
if dataset.isna().sum().sum() != 0:
    print('데이터에 결측치가 있습니다.')
    raise ValueError

In [ ]:
# Pearson 상관계수를 통해 유입량과의 상관관계가 높은 변수만 선택
corr_with_target = dataset.corr()[target_column]
#filtered_columns = dataset.columns[abs(corr_with_target) >= 0.35]# 상관계수의 절대값이 0.35보다 큰 변수만 선택하는 기능, 필요시 주석 해제
#dataset=dataset[filtered_columns]

# 선택된 변수들의 상관계수를 시각화
dataset_4EDA = dataset.copy()
dataset_4EDA.columns=['10분 최다 강수량', '1시간 최다강수량', '일강수량1', '9-9강수', '일강수량1_lag1','일강수량1_lag2', '일강수량2', '유입량', '일강수량2_lag1', '일강수량2_lag2']
dataset_4EDA=dataset_4EDA[['유입량','10분 최다 강수량', '1시간 최다강수량', '9-9강수', '일강수량1', '일강수량1_lag1','일강수량1_lag2', '일강수량2',  '일강수량2_lag1', '일강수량2_lag2']]
plt.figure(figsize=(20, 10))
sns.heatmap(dataset_4EDA.corr(), annot=True, fmt='.2f', linewidths=2)
del dataset_4EDA

4. 딥러닝 모델 생성

In [ ]:
###########################################################################
TS=float # 딥러닝 학습용데이터와 테스트 데이터 비율 70:30일경우 ex:0.3
learning_rate = float # 학습률 (작을수록 학습을 세밀하게 조정) ex:0.001
batch_size = int # 배치 사이즈(한번에 학습할 데이터 양) ex:32
epochs = int # 학습 횟수 ex:1000
hidden_size1 = int # 첫번째 은닉층 노드 수 ex:64
hidden_size2 = int # 두번째 은닉층 노드 수 ex:32
###########################################################################
# 학습 데이터와 테스트 데이터로 분할
X = dataset.drop(columns=[target_column])
y = dataset[[target_column]].copy()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=TS, shuffle=False)
input_size = x_train.shape[1]
output_size = y_train.shape[1]

# 데이터 정규화 (정규화란?: https://m.blog.naver.com/PostView.nhn?blogId=youji4ever&logNo=221384104383&proxyReferer=https:%2F%2Fwww.google.com%2F)
scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
scaler.fit(x_train)
y_scaler.fit(y_train)

# 데이터를 머신러닝 모델에 입력하기 위해 텐서로 변환 (Tensor란?: https://rekt77.tistory.com/102)
train_dataset = TensorDataset(torch.FloatTensor(scaler.transform(x_train)), torch.FloatTensor(y_scaler.transform(y_train)))
val_loader = DataLoader(TensorDataset(torch.FloatTensor(scaler.transform(x_test)), torch.FloatTensor(y_scaler.transform(y_test))),\
    batch_size=batch_size, shuffle=False, num_workers=0)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

In [ ]:
# LSTM 모델 생성
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(LSTM, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.lstm1 = nn.LSTM(input_size, hidden_size1, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size1, hidden_size2, batch_first=True)
        self.linear = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = x.unsqueeze(1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.linear(x[:, -1, :])
        return x

# BiLSTM 모델 생성
class BILSTM(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(BILSTM, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.lstm1 = nn.LSTM(input_size, hidden_size1, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(hidden_size1*2, hidden_size2, batch_first=True, bidirectional=True)
        self.linear = nn.Linear(hidden_size2*2, output_size)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = x.unsqueeze(1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.linear(x[:, -1, :])
        return x

# GRU 모델 생성
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(GRU, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.gru1 = nn.GRU(input_size, hidden_size1, batch_first=True)
        self.gru2 = nn.GRU(hidden_size1, hidden_size2, batch_first=True)
        self.linear = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = x.unsqueeze(1)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = self.linear(x[:, -1, :])
        return x

In [ ]:
# 모델을 이용하여 학습을 위한 기능 제작

def train_model(model, optimizer, criterion, train_loader, val_loader, epochs, device, patience=10):
    best_loss = 100
    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for i, data in enumerate(val_loader, 0):
                inputs, labels = data[0].to(device), data[1].to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model.state_dict()
            patience_count = 0
        else:
            patience_count += 1
            if patience_count > patience:
                model.load_state_dict(best_model)
                print('Early stopping')
                break
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}')
    return model

# 모델과 학습기를 이용한 학습
modelBI= BILSTM(input_size, hidden_size1, hidden_size2, output_size)
optimizer = optim.Adam(modelBI.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
modelBI = train_model(modelBI, optimizer, criterion, train_loader, val_loader, epochs, device)

modelLS= LSTM(input_size, hidden_size1, hidden_size2, output_size)
optimizer = optim.Adam(modelLS.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
modelLS = train_model(modelLS, optimizer, criterion, train_loader, val_loader, epochs, device)

modelGR= GRU(input_size, hidden_size1, hidden_size2, output_size)
optimizer = optim.Adam(modelGR.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
modelGR = train_model(modelGR, optimizer, criterion, train_loader, val_loader, epochs, device)

In [ ]:
# Tensor를 일반 데이터로 변환 및 역정규화를 통해 실제값으로 변환
modelBI.eval()
modelLS.eval()
modelGR.eval()
test_results_BI = []
test_results_LS = []
test_results_GR = []
for x, y in val_loader:
    x = x.to(device)
    test_results_BI.append(modelBI(x))
    test_results_LS.append(modelLS(x))
    test_results_GR.append(modelGR(x))
test_results_BI = torch.cat(test_results_BI)
test_results_LS = torch.cat(test_results_LS)
test_results_GR = torch.cat(test_results_GR)
test_results_BI = y_scaler.inverse_transform(test_results_BI.cpu().detach().numpy())
test_results_LS = y_scaler.inverse_transform(test_results_LS.cpu().detach().numpy())
test_results_GR = y_scaler.inverse_transform(test_results_GR.cpu().detach().numpy())
x_train_tensor = torch.FloatTensor(scaler.transform(x_train)).to(device)
train_results_BI = modelBI(x_train_tensor)
train_results_LS = modelLS(x_train_tensor)
train_results_GR = modelGR(x_train_tensor)
train_results_BI = y_scaler.inverse_transform(train_results_BI.cpu().detach().numpy())
train_results_LS = y_scaler.inverse_transform(train_results_LS.cpu().detach().numpy())
train_results_GR = y_scaler.inverse_transform(train_results_GR.cpu().detach().numpy())

# 학습 및 테스트 데이터의 예측값을 하나의 데이터로 합침
results_BI = np.concatenate((train_results_BI.flatten(), test_results_BI.flatten()), axis=0)
results_LS = np.concatenate((train_results_LS.flatten(), test_results_LS.flatten()), axis=0)
results_GR = np.concatenate((train_results_GR.flatten(), test_results_GR.flatten()), axis=0)

# 결과를 데이터프레임(엑셀시트)으로 변환
datares = pd.DataFrame()
datares[target_column] = dataset[target_column]
datares['Inflow_BI-LSTM'] = results_BI
datares['Inflow_LSTM'] = results_LS
datares['Inflow_GRU'] = results_GR
datares = datares.applymap(lambda x: x if x > 0 else 0)

In [ ]:
# 결과를 엑셀형식으로 확인
datares

5. 결과 검증

In [ ]:
# 수문학 모델의 성능을 평가하기 위한 함수들

def calc_MAE(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def calc_RMSE(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

def calc_MAPE(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def calc_NSE(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def calc_r2(y_true, y_pred):
    mean_t = np.mean(y_true)
    mean_p = np.mean(y_pred)
    return 1 - (np.sum((y_true - mean_t) * (y_pred - mean_p)) / np.sqrt(np.sum((y_true - mean_t)**2) * np.sum((y_pred - mean_p)**2)))**2

def calc_pe(y_true, y_pred):
    return ((y_true - y_pred) / y_true) * 100

def calc_pbias(y_true, y_pred):
    return 100 * np.sum(y_true - y_pred) / np.sum(y_true)

def calc_rsr(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

def calc_mNSE(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def calc_md(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

def calc_rd(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_pred))

def calc_cp(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred)**2) / np.sum((np.abs(y_true - np.mean(y_true)) + np.abs(y_pred - np.mean(y_pred)))**2))

def calc_r(y_true, y_pred):
    return np.sum(y_true * y_pred) / np.sqrt(np.sum(y_true**2) * np.sum(y_pred**2))

def calc_bR2(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

def calc_rNSE(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))

def calc_kge(y_true, y_pred):
  r = np.corrcoef(y_true, y_pred)[0, 1]
  return 1 - np.sqrt((r - 1)**2 + (np.std(y_pred) / np.std(y_true) - 1)**2 + (np.mean(y_pred) / np.mean(y_true) - 1)**2)

def calc_ve(y_true, y_pred):
    return np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

def calc_all(y_true, y_pred):
    res_dict = {}
    res_dict['MAE'] = calc_MAE(y_true, y_pred)
    res_dict['RMSE'] = calc_RMSE(y_true, y_pred)
    res_dict['NSE'] = calc_NSE(y_true, y_pred)
    #res_dict['r2'] = calc_r2(y_true, y_pred)
    #res_dict['pbias'] = calc_pbias(y_true, y_pred)
    #res_dict['rsr'] = calc_rsr(y_true, y_pred)
    #res_dict['mNSE'] = calc_mNSE(y_true, y_pred)
    #res_dict['md'] = calc_md(y_true, y_pred)
    #res_dict['rd'] = calc_rd(y_true, y_pred)
    #res_dict['cp'] = calc_cp(y_true, y_pred)
    #res_dict['r'] = calc_r(y_true, y_pred)
    #res_dict['bR2'] = calc_bR2(y_true, y_pred)
    #res_dict['rNSE'] = calc_rNSE(y_true, y_pred)
    #res_dict['kge'] = calc_kge(y_true, y_pred)
    #res_dict['ve'] = calc_ve(y_true, y_pred)
    return res_dict

result_with_metric = {}
result_with_metric['Inflow_BI-LSTM'] = calc_all(datares['Inflow'], datares['Inflow_BI-LSTM'])
result_with_metric['Inflow_LSTM'] = calc_all(datares['Inflow'], datares['Inflow_LSTM'])
result_with_metric['Inflow_GRU'] = calc_all(datares['Inflow'], datares['Inflow_GRU'])
df_results = pd.DataFrame(result_with_metric)
# 결과를 엑셀형식으로 확인. 평가항목은 MAE, RMSE, NSE
df_results

In [ ]:
# 결과를 그래프로 시각화
import plotly.graph_objects as go
fig=go.Figure()
#add pred data
fig.add_trace(go.Scatter(x=datares.index, y=datares['Inflow_BI-LSTM'], mode='lines', name='Inflow_BI-LSTM', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=datares.index, y=datares['Inflow_LSTM'], mode='lines', name='Inflow_LSTM', line=dict(color='red')))
fig.add_trace(go.Scatter(x=datares.index, y=datares['Inflow_GRU'], mode='lines', name='Inflow_GRU', line=dict(color='green')))
#add real data
fig.add_trace(go.Scatter(x=datares.index, y=datares['Inflow'], mode='lines', name='Inflow', line=dict(color='black')))
